In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from datetime import datetime
import geopandas as gpd
import os
from shapely.geometry import Polygon, LineString, Point
from fiona.crs import from_string

In [ ]:
df = []

In [ ]:
df = pd.read_csv('daegu_service_pcell_time_pop_202101.csv', sep = '|')  # sep는 엑셀파일에서 하나의 셀에 '|'을 기준으로 모든 속성과 값이 들어 있을 때 
                                                                        # 분리하여 각 값을 하나의 셀에 넣어 만들어주는 역할을 한다.

In [ ]:
df.dtypes       # 먼저 각 속성의 데이터타입을 확인한다.

In [ ]:
df['HCODE'] = df['HCODE'].apply(str)  # 속성 HCODE의 속성을 문자열로 바꿔준다. 왜냐하면 특정문자를 포함하는 행만 추출하기 위해서 
                                      # 숫자타입으로는 조건을 줄 수 없어서 문자형태로 고쳐준 것이다.
                                      # 문자타입으로 고쳐진 속성은 특정문자를 포함하는 행을 추출할 수 있다.

In [ ]:
df = df[df['HCODE'].str.contains('27290')]     # 문자열 27290을 포함하는 행 추출
HCODE['HCODE'].unique()                        # 속성의 유니크한 값을 추출

In [ ]:
df['STD_YMD'] = pd.to_datetime(df['STD_YMD'], format = '%Y%m%d')  # ['STD_YMD']속성을 데이터타임의 형태로 바꾸고, 연도가 4자리
                                                                  # 월이 2자리 일이 2자리인 형태로 나타내라는 의미

In [ ]:
df['WKDY_CD'] = df.STD_YMD.dt.dayname()    # weekday code 속성을 새로 만들고 STD_YMD를 기준으로 문자형태의 요일을 추출하라
df['WKDY_CD'] = df['WKDY_CD'].str[:3]      # ['WKDY_CD'] 속성에서 0,1,2에 해당하는 문자만 추출하라 

In [ ]:
ex1 = pd.melt(df,id_vars = ['STD_YMD','X_COORD','Y_COORD','HCODE','WKDY_CD'], value_vars = ['W_T_00','W_T_00','W_T_00','W_T_00','W_T_00','W_T_00','W_T_00','W_T_00','W_T_23']).sort_values(by = ['STD_YMD','X_COORD','Y_COORD'])
# id_vars는 고정값 , vaule_vars는 세로로 붙여질 값들, sort_values 정렬기준

In [ ]:
ex1.rename(columns = {'value' : 'WPOP'}, inplace = True)    # value라는 속성의 열속성 이름을 WPOP로 변경하라
ex1.rename(columns = {'variable' : 'TIME'}, inplace = True) # variable라는 속성의 열속성 이름을 TIME로 변경하라

In [ ]:
ex2 = pd.melt(df,id_vars = ['STD_YMD','X_COORD','Y_COORD','HCODE','WKDY_CD'], value_vars = ['V_T_00','V_T_00','V_T_00','V_T_00','V_T_00','W_T_00','W_T_00','W_T_00','W_T_23']).sort_values(by = ['STD_YMD','X_COORD','Y_COORD'])
ex2.rename(columns = {'value' : 'VPOP'}, inplace = True)
ex2 = ex2[['VPOP']]

In [ ]:
ex3 = pd.melt(df,id_vars = ['STD_YMD','X_COORD','Y_COORD','HCODE','WKDY_CD'], value_vars = ['H_T_00','H_T_00','H_T_00','H_T_00','H_T_00','W_T_00','W_T_00','W_T_00','W_T_23']).sort_values(by = ['STD_YMD','X_COORD','Y_COORD'])
ex3.rename(columns = {'value' : 'HPOP'}, inplace = True)
ex3 = ex3[['HPOP']]

In [ ]:
final = pd.concat([ex1, ex2, ex3], axis = 1) # 열(옆에)로 붙여라 

In [ ]:
df = final

In [ ]:
df = df[['STD_YMD','X_COORD','Y_COORD','HCODE','WKDY_CD','TIME','HPOP','WPOP','VPOP']]  # 원하는 열 순서대로 만들기

In [ ]:
df['TIME'] = df['TIME'].str[4:]   # TIME 속성의 값의 4번쨰 부터 끝까지만 추출해서 다시 TIME열에 넣기 

In [ ]:
df['WKDY_CD'] = df['WKDY_CD'].str.upper()

In [ ]:
df['HCODE'] = df['HCODE'].apply(str) 
df['HCODE'] = df['HCODE'].replace(['2729058500','2729060200','2729057700'......],['22000','23000','24000'......]) 
df['HCODE'].unique()

In [ ]:
df['TIME'] = df['TIME'].apply(str)
df['STD_YMD'] = df['STD_YMD'].apply(str)

In [ ]:
df['STD_YMD'] = df['STD_YMD'].str[:4] + df['STD_YMD'].str[5:7]   #202101
df['STD_YMD'] = pd.to_numeric(df['STD_YMD'])

In [ ]:
sample_csv = df

In [ ]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

In [ ]:
sample_shp = gpd.GeoDataFrame.from_file('C:/Users/빅데이터활용센터/next/nlsp_031001001.shp', encoding='utf-8')
sample_shp.crs = epsg5179

In [ ]:
sample_csv['xcode'] = sample_csv['xcode'].astype(float)
sample_csv['ycode'] = sample_csv['ycode'].astype(float)
sample_csv['geometry'] = sample_csv.apply(lambda row : Point([row['xcode'], row['ycode']]), axis=1)
sample_csv = gpd.GeoDataFrame(sample_csv, geometry='geometry', crs = epsg5179)

In [ ]:
result = gpd.sjoin(sample_csv, sample_shp, how='left', predicate="intersects")
result = result[['STD_YMD','gid','HCODE','WKDY_CD','TIME','HPOP','WPOP','VPOP']]
result.rename(columns = {'gid' : 'INDEX_ID'}, inplace = True)

In [ ]:
result = result.groupby(['STD_YMD','INDEX_ID','HCODE','WKDY_CD','TIME']).agg('sum')

In [ ]:
result

In [ ]:
result.to_csv('202101(최종).csv', encoding = 'ansi', index = True, sep = '|')